In [1]:
# Add path candidates to sys.path to import utility / helper modules
import sys
sys.path.append('/workspace')

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Quick testing with existing data files

"""
from contextlib import redirect_stdout
from difflib import unified_diff
from io import StringIO
from pathlib import Path
from re import findall


# Run main() with data files and prints out differences between
# expected data and output
def run(main):
    # Lookup data file with directory name
    cwd = Path.cwd()
    data = cwd / f'{cwd.name}.dat'

    # If <working_directory_name>.dat is not found
    # then find any first data file in working directory
    if not data.exists():
        data = None
        for child in cwd.iterdir():
            if child.name.endswith('.dat'):
                data = child
                break

    # If there is no data file, do panic
    if data is None:
        raise FileNotFoundError('.dat file is not found')

    # Read data file and mock stdin with StringIO
    with data.open(mode='r', encoding='utf-8') as fp:
        data = fp.read()
        input = findall(r'^\[in\]([\d\w\s]*)\[out\]', data)[0]
        ifs = StringIO(input.strip())
        expected = findall(r'\[out\]([\s\S]*)$', data)[0]

    # Run program and with redirection of stdout to buffer
    with StringIO() as ofs, redirect_stdout(ofs):
        main(ifs.readline)
        result = ofs.getvalue()

    # Print differences
    sys.stdout.writelines(
        unified_diff(
            expected.lstrip().splitlines(keepends=True),
            result.lstrip().splitlines(keepends=True),
            fromfile='expected',
            tofile='result',
        )
    )

In [4]:
"""
Version 1: TIMEOUT
- used methods: DP

"""
from functools import lru_cache


# Solution
def solution(sequence, k):
    n = len(sequence)
    result = []

    # Same as algospot-LIS
    @lru_cache(maxsize=n)
    def lis(start):
        ret = 1
        for next in range(start + 1, n):
            if start == -1 or sequence[next] > sequence[start]:
                ret = max(ret, 1 + lis(next))

        return ret

    # Count of LIS from :start where :lis(:start) == 1 + :lis(:next)
    @lru_cache(maxsize=n)
    def count(start):
        if lis(start) == 1:
            return 1

        ret = 0
        for next in range(start + 1, n):
            if (start == -1 or sequence[next] > sequence[start]) \
                    and lis(next) + 1 == lis(start):
                ret += count(next)

        return ret

    # Reconstruct k-th LIS sequence from :lis and :count
    def reconstruct(start, skip):
        if start != -1:
            result.append(sequence[start])

        candidates = []
        for next in range(start + 1, n):
            if (start == -1 or sequence[next] > sequence[start]) \
                    and lis(next) + 1 == lis(start):
                candidates.append((sequence[next], next))

        candidates.sort()
        for (_, next) in candidates:
            c = count(next)
            if c <= skip:
                skip -= c
            else:
                return reconstruct(next, skip)

    reconstruct(-1, k - 1)
    return result


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        _, k = map(int, rl().split())
        sequence = list(map(int, rl().split()))
        result = solution(sequence, k)
        print(len(result))
        print(' '.join(map(str, result)))


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    # IPython env
    run(main)

In [5]:
"""
Testing

"""
import cProfile
import traceback
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    (([1, 9, 7, 4, 2, 6, 3, 11, 10], 2),
     [1, 2, 3, 11]),
    (([2, 1, 4, 3, 6, 5, 8, 7], 4),
     [1, 3, 6, 8]),
    (([5, 6, 7, 8, 1, 2, 3, 4], 2),
     [5, 6, 7, 8]),
    ((list(range(500)), 1),
     list(range(500))),
    ((list(reversed(range(500))), 1),
     [0]),
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        traceback.print_exc()
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00033570] 'solution([1, 9, 7, 4, 2, 6, 3, 11, 10], 2)' → '[1, 2, 3, 11]'
Case 1 PASS

[0.00013460] 'solution([2, 1, 4, 3, 6, 5, 8, 7], 4)' → '[1, 3, 6, 8]'
Case 2 PASS

[0.00010360] 'solution([5, 6, 7, 8, 1, 2, 3, 4], 2)' → '[5, 6, 7, 8]'
Case 3 PASS

[0.15711920] 'solution([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,... → '[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...
Case 4 PASS

[0.01887930] 'solution([499, 498, 497, 496, 495, 494, 493, 492, 491, 490, 489, 488, 487, 486, 485, 484, 483, 482, 481, 480, 479, 478,... → '[0]'
Case 5 PASS


All test(s) have passed
